In [1]:
using Distributed

addprocs(6)

6-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7

In [13]:
@everywhere using Lasso
using NPZ
@everywhere using Statistics
using PyCall
@everywhere using SparseArrays

In [14]:
py"""
import scipy.sparse as sp

def spload(x):
    return sp.load_npz(x)
"""

In [15]:
G = py"spload"("Data/Gcsc.npz")
G = SparseMatrixCSC(G.shape[1], G.shape[2], G.indptr .+ 1, G.indices .+ 1, G.data)

7646×20122 SparseMatrixCSC{Float64,Int64} with 34965654 stored entries:
  [1   ,     1]  =  -5.19568
  [2   ,     1]  =  -4.83212
  [3   ,     1]  =  -4.46621
  [4   ,     1]  =  -4.09542
  [5   ,     1]  =  -3.71781
  [6   ,     1]  =  -3.33932
  [7   ,     1]  =  -2.94609
  [8   ,     1]  =  -2.58125
  [9   ,     1]  =  -2.19924
  [10  ,     1]  =  -1.81655
  [11  ,     1]  =  -1.40428
  [12  ,     1]  =  -1.04932
  ⋮
  [7634, 20122]  =  -0.0377013
  [7635, 20122]  =  -0.0565704
  [7636, 20122]  =  -0.0702979
  [7637, 20122]  =  -0.0555521
  [7638, 20122]  =  -0.0547308
  [7639, 20122]  =  -0.0672549
  [7640, 20122]  =  -0.0348468
  [7641, 20122]  =  -0.0510785
  [7642, 20122]  =  -0.0623164
  [7643, 20122]  =  -0.0498602
  [7644, 20122]  =  -0.0626302
  [7645, 20122]  =  -0.0332227
  [7646, 20122]  =  -0.0332705

In [16]:
@everywhere G = $G

In [17]:
@everywhere function fit_model(G,dv,i)
    dn = std(dv)
    dv ./= dn
    time = @elapsed path = fit(LassoPath, G, dv, λminratio = 1e-4, standardize=false, intercept=false) 
    λmin = path.λ[argmin(aicc(path))]
    m = coef(path; select=MinAICc())
    println("Fitted data $i at $(now()) in $time s on worker $(myid())")
    flush(stdout)
    return Dict("i"=>i,"reg"=>λmin, "dn"=>dn, "m"=>m)
end

In [18]:
d = npzread("Data/wvtdata.npy")
d = [(i, d[i,:]) for i = 1:size(d,1)];

In [ ]:
results = pmap(x->fit_model(G, x[2], x[1]), d)